**Task 07: Querying RDF(s)**

In [10]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

Read the RDF file

In [11]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N73519badab9f41d28b9a4ef29bc1a3e0 (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

RDFlib

We list all the direct subclasses of Person

In [26]:
print('Direct subclasses of person:')
ns = Namespace("http://somewhere#")
for s,p,o in g.triples((None,RDFS.subClassOf,ns.Person)):
  print(s)


Direct subclasses of person:
http://somewhere#Researcher


We list now all the classes in the hierarchy

In [25]:
print('All the subclasses of person:')
ns = Namespace("http://somewhere#")
print(ns.person)
for s,p,o in g.triples((None,RDFS.subClassOf,ns.Person)):
  print(s)
  for s1,p1,o1 in g.triples((None,RDFS.subClassOf,s)):
    print(s1)
  

All the subclasses of person:
http://somewhere#person
http://somewhere#Researcher
http://somewhere#PhDstudent


SPARQL

We list first all the direct subclasses (as we did in RDFlib) and then all the subclasses in the hierarchy

In [22]:

from rdflib.plugins.sparql import prepareQuery
#Direct subclasses
q1_1 = prepareQuery('''
  SELECT 
    ?Subclass
  WHERE { 
    ?Subclass rdfs:subClassOf ns:Person. 
  }
  ''',
  initNs = { "ns": ns}
)

print('\nJust subclasses one level below in the hierarchy\n')
for r in g.query(q1_1):
  print(r)

#All subclasses
q1 = prepareQuery('''
  SELECT 
    ?Subclass
  WHERE { 
    ?Subclass rdfs:subClassOf* ns:Person. 
  }
  ''',
  initNs = { "ns": ns}
)

print('\nAll the subclasses in the hierarchy\n')
for r in g.query(q1):
  print(r)




Just subclasses one level below in the hierarchy

(rdflib.term.URIRef('http://somewhere#Researcher'),)

All the subclasses in the hierarchy

(rdflib.term.URIRef('http://somewhere#Person'),)
(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


RDFlib

In [14]:
#We want to store all the results of this query in a list for task 7.3
individuals=[]

print("Individuals of class person:")

for s,p,o in g.triples((None, RDF.type, ns.Person)):
  print(s)
  individuals.append(s)

print("\nIndividuals of subclasses of person: ")  
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for x,y,z in g.triples((None, RDF.type, s)):
    print(x)
    individuals.append(x)


Individuals of class person:
http://somewhere#SaraJones
http://somewhere#JohnSmith

Individuals of subclasses of person: 
http://somewhere#JaneSmith


SPAQRL

In [15]:

q2 = prepareQuery('''
  SELECT 
    ?s 
  WHERE { 
    ?s rdf:type/rdfs:subClassOf* ns:Person.
  }
  ''',
  initNs = { "ns": ns}
)


for r in g.query(q2):
  print(r)


(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


RDFlib

In [18]:
#We use the list constructed in the previous task
c=0
for i in individuals:
  for x,y,z in g.triples((i,None,None)):
    c=c+1
    print(i,y,z)
print(c)

http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewhere#h

SPARQL

In [17]:

q3 = prepareQuery('''
  SELECT 
    ?s ?p ?o
  WHERE { 
    ?s rdf:type/rdfs:subClassOf* ns:Person.
    ?s ?p ?o
  }
  ''',
  initNs = { "ns": ns}
)

print(len(g.query(q3)))
for r in g.query(q3):
  print(r)


12
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'), rdflib.term.Literal('Sara', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'), rdflib.term.Literal('Sara Jones', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'), rdflib.term.Literal('Jones', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://somewhere#Person'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-r